# 挂载到云端硬盘，使得可以读入下载数据

In [8]:
import os
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/My Drive"

os.chdir(path)
os.listdir(path)

Mounted at /content/drive


['pytorch-handbook']

# PyTorch 基础 :数据的加载和预处理
PyTorch通过`torch.utils.data`对一般常用的`数据加载`进行了封装，可以很容易地实现`多线程数据预读`和`批量加载`。

并且`torchvision`已经预先实现了`常用图像数据集`，包括前面使用过的CIFAR-10，ImageNet、COCO、MNIST、LSUN等数据集，可通过`torchvision.datasets`方便的调用

In [1]:
# 首先要引入相关的包
import torch
#打印一下版本
torch.__version__

'1.10.0+cu111'

## Dataset
Dataset是一个抽象类，为了能够方便的读取，需要`将要使用的数据包装为Dataset类`。
自定义的Dataset需要继承它并且实现两个成员方法：
1. `__getitem__()` 该方法定义用索引(`0` 到 `len(self)`)获取一条数据或一个样本，`__getitem__()`可以使我们通过索引来找到一个实例

2. `__len__()` 该方法返回数据集的总长度

PS：不要忘了应该要初始化这个类的属性，`def __init__`

下面我们使用kaggle上的一个竞赛[bluebook for bulldozers](https://www.kaggle.com/c/bluebook-for-bulldozers/data)自定义一个数据集，为了方便介绍，我们使用里面的数据字典来做说明（因为条数少）

In [2]:
#引用
from torch.utils.data import Dataset
import pandas as pd

In [3]:
## 定义一个数据集
class BulldozerDataset(Dataset):
  ## BulldozerDataset类继承Dataset
  def __init__(self,csv_file):
    ## 初试化数据集类，属性为一个csv文件
    self.df=pd.read_csv(csv_file)

  def __len__(self):
    ## __len__是要计算数据集的总长度
    return len(self.df)
  
  def __getitem__(self,idx):
    ## __getitem__帮助我们通过索引来返回一个实例
    return self.df.iloc[idx].SalePrice ## 这里可能不需要用 .iloc方法也可以，因为直接用Index来索引


至此，我们的数据集已经定义完成了，我们可以实例化一个对象访问他

In [12]:
print(os.listdir())
ds_demo= BulldozerDataset('/content/drive/MyDrive/pytorch-handbook/chapter2(pytorch basic)/median_benchmark.csv') ## 方法，右边那个文件夹显示，然后去最上面找路径

['pytorch-handbook']


尝试用一下我们之前定义的`__len__()`和`__getitem__()`方法

In [14]:
print(ds_demo.__len__()) ## 尝试使用 .__len__()方法
print(ds_demo[0]) ## .__getitem__()帮助我们通过索引你来访问一个实例，很类似 .iloc[某行,]

11573
24000.0


自定义的数据集已经创建好了，下面我们使用官方提供的数据载入器，读取数据
## Dataloader
`DataLoader`为我们提供了`对Dataset的读取操作`，可以说,`DataLoader`专门用来读取`Dataset`

常用参数有：

- batch_size(每个batch的大小)

- shuffle(是否进行shuffle操作)

- num_workers(加载数据的时候使用几个子进程)。

下面做一个简单的操作：

In [16]:
dl=torch.utils.data.DataLoader(ds_demo,batch_size=10,shuffle=True,num_workers=10)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


`DataLoader`返回的是一个可迭代对象，我们可以使用迭代器分次获取数据。即，DataLoader相当于帮我们缓读了数据，并且做了分batch，shuffle等操作。

In [17]:
idata=iter(dl) ## 获得其中一个batch
print(idata.__len__()) ## 其中一个batch的长度
print(next(idata)) ## next batch

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


1158
tensor([24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000.,
        24000.], dtype=torch.float64)


常见的用法是使用`for循环`对其进行遍历

In [18]:
for i, data in enumerate(dl): ## enumerate针对一个可以迭代的对象，dl是之前定义的Dataload对象，是个迭代器，其中每一个对象都是一个batch
    print(i,data) ## 打印这个batch
    # 为了节约空间，这里只循环一遍
    break

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 10 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


0 tensor([24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000., 24000.,
        24000.], dtype=torch.float64)


我们已经可以通过`dataset定义数据集`，并使用`Datalorder载入`和遍历数据集，除了这些以外，PyTorch还提供能torcvision的计算机视觉扩展包
## torchvision 
`torchvision` 是PyTorch中专门用来`处理图像`的库，PyTorch官网的安装教程中最后的pip install torchvision 就是安装这个包。

### torchvision.datasets
torchvision.datasets 可以理解为PyTorch团队自定义的dataset，这些dataset帮我们提前处理好了很多的图片数据集，我们拿来就可以直接使用：
- MNIST
- COCO
- Captions
- Detection
- LSUN
- ImageFolder
- Imagenet-12
- CIFAR
- STL10
- SVHN
- PhotoTour

我们可以直接使用，示例如下：

In [ ]:
import torchvision.datasets as datasets
trainset = datasets.MNIST(root='./data', # 表示MNIST数据的加载的目录
                                      train=True,  # 表示是否加载数据库的训练集，false的时候加载测试集
                                      download=True, # 表示是否自动下载 MNIST 数据集
                                      transform=None) # 表示是否需要对数据进行预处理，none为不进行预处理


### torchvision.models
torchvision不仅提供了常用图片数据集，还提供了`训练好`的模型，可以加载之后，直接使用，或者在进行`迁移学习`

`torchvision.models`模块的 子模块中包含以下模型结构。
- AlexNet
- VGG
- ResNet
- SqueezeNet
- DenseNet

就类似torch.nn.Module里面有fully-connected layer，CONV layer一样。


In [ ]:
#我们直接可以使用训练好的模型，当然这个与datasets相同，都是需要从服务器下载的
import torchvision.models as models
resnet18 = models.resnet18(pretrained=True) ## pretrained，预训练好的

### torchvision.transforms
`transforms`模块提供了一般的图像转换操作类，用作`数据处理`和`数据增强`

In [ ]:
from torchvision import transforms as transforms
transform = transforms.Compose([
    transforms.RandomCrop(32, padding=4),  #先四周填充0，在把图像随机裁剪成32*32
    transforms.RandomHorizontalFlip(),  #图像一半的概率翻转，一半的概率不翻转
    transforms.RandomRotation((-45,45)), #随机旋转
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.229, 0.224, 0.225)), #R,G,B每层的归一化用到的均值和方差
])

肯定有人会问：(0.485, 0.456, 0.406), (0.2023, 0.1994, 0.2010) 这几个数字是什么意思？

图像每个像素都有RGB值，0-225，构成一个彩色图像，三个通道，如果只有一个通道那么就是一个灰度图像。对于每个channel进行归一化需要一个均值和方差，上面那两个三元组对应了每一个通道的均值和方差，具体为什么是这些参数看下面那个链接

官方的这个帖子有详细的说明:
https://discuss.pytorch.org/t/normalization-in-the-mnist-example/457/21

这些都是根据ImageNet训练的归一化参数，可以直接使用，我们认为这个是固定值就可以